# Submit Predictions

This notebook shows how to prepare a submission of your model's predictions on the test data for the computation.

In [10]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))

from pathlib import Path
from natsort import natsorted
import numpy as np
import torch 
import pandas as pd

# LEFT_TO_RIGHT_HAND = [6, 5, 4, 3, 2, 1, 0, 7]


## Load pre-trained model

This code loads the pre-trained baseline model - might be different for your model.

In [11]:
import wandb
from os.path import join
import yaml
from helpers.predict_utils import Config
from helpers.models import TimeSeriesRegressorWrapper
import torch


device = torch.device('cpu')
wandb_mode = 'disabled'
wandb_project = 'hvat'
wandb_name = 'SL-2'
with open(join('data', 'A_1', 'configs', f'hvat.yaml'), 'r') as file:
    wandb_config = yaml.safe_load(file)
    config = Config(wandb_config)
with wandb.init(mode=wandb_mode, project=wandb_project, name=wandb_name, config=config):
    config = wandb.config

model = TimeSeriesRegressorWrapper(device=device, input_size=len(config.features), output_size=len(config.targets), **config)
model.to(device)

model.load('/Users/jg/projects/biomech/biomech_PCP/data/hvat/models/SL-2.pt')

inv_model = TimeSeriesRegressorWrapper(device=device, input_size=len(config.features), output_size=len(config.targets), **config)
inv_model.to(device)

inv_model.load('/Users/jg/projects/biomech/biomech_PCP/data/hvat/models/SL-2_inverted.pt')           

### Save `submission.cvs` file

This code shows how the data was prepare and downsampled during inference. Make sure that your data is processed similarly!

In [12]:
DATA_PATH = Path(r"data/hvat")
test_data_name = 'fedya_tropin_standart_elbow_left'  # should match `test_dataset_list` used to train the model


data_folder = DATA_PATH / "amputant" / "left" / test_data_name / "preproc_angles" / "submit"
all_paths = natsorted(data_folder.glob('*.npz'))
print(f'Found {len(all_paths)} samples in {data_folder}')


Found 72 samples in data/hvat/amputant/left/fedya_tropin_standart_elbow_left/preproc_angles/submit


In [14]:

pred_list = []

# loop over each trial
for i, p in enumerate(all_paths):
    # get EMG data 
    sample = np.load(p)
    myo = sample['data_myo']

    padded_myo = myo

    preds = model.predictNPonly(torch.tensor(padded_myo, device=device, dtype=torch.float32).unsqueeze(0)).squeeze().cpu().numpy()
    inv_preds = inv_model.predictNPonly(torch.tensor(np.flip(padded_myo).copy(), device=device, dtype=torch.float32).unsqueeze(0)).squeeze().cpu().numpy()
    inv_preds = np.flip(inv_preds, axis=0).copy()
    preds_downsampled = preds[::8]
    inv_preds_downsampled = inv_preds[::8]
    combined_preds = preds_downsampled.copy()
    combined_preds[:50] = inv_preds_downsampled[:50]
    combined_preds[50:-50] = (preds_downsampled[50:-50] + inv_preds_downsampled[50:-50]) / 2
    print(f"Completed {i+1}/{len(all_paths)}. Loaded data: {myo.shape} - padded to: {padded_myo.shape} - predictions {preds.shape} - downsampled to: {preds_downsampled.shape}")
    pred_list.append(combined_preds)

pred_cat = np.concatenate(pred_list, axis=0)
df = pd.DataFrame(pred_cat)
df.head()
print(df.shape)

Completed 1/72. Loaded data: (3721, 8) - padded to: (3721, 8) - predictions (3721, 20) - downsampled to: (466, 20)
Completed 2/72. Loaded data: (3725, 8) - padded to: (3725, 8) - predictions (3725, 20) - downsampled to: (466, 20)
Completed 3/72. Loaded data: (3724, 8) - padded to: (3724, 8) - predictions (3724, 20) - downsampled to: (466, 20)
Completed 4/72. Loaded data: (3724, 8) - padded to: (3724, 8) - predictions (3724, 20) - downsampled to: (466, 20)
Completed 5/72. Loaded data: (3729, 8) - padded to: (3729, 8) - predictions (3729, 20) - downsampled to: (467, 20)
Completed 6/72. Loaded data: (3726, 8) - padded to: (3726, 8) - predictions (3726, 20) - downsampled to: (466, 20)
Completed 7/72. Loaded data: (3724, 8) - padded to: (3724, 8) - predictions (3724, 20) - downsampled to: (466, 20)
Completed 8/72. Loaded data: (3728, 8) - padded to: (3728, 8) - predictions (3728, 20) - downsampled to: (466, 20)
Completed 9/72. Loaded data: (3724, 8) - padded to: (3724, 8) - predictions (372

In addition to the predictions, your data should also include a sample id column.

In [15]:
df.insert(0, "sample_id", range(1, 1 + len(df)))

Finally, save to a CSV file. This is what you'll upload to Kaggle for the competition.

In [16]:
os.makedirs('data/hvat/submissions', exist_ok=True)
df.to_csv('data/hvat/submissions/submit_file_4.csv', index=False)

In [ ]:
df.shape